In [1]:
from pynvml import *

nvmlInit()
vram = nvmlDeviceGetMemoryInfo(nvmlDeviceGetHandleByIndex(1)).free/1024.**2
print('GPU1 Memory: %dMB' % vram)
if vram < 8000:
    raise Exception('GPU Memory too low')
nvmlShutdown()

GPU1 Memory: 11172MB


In [2]:
import os
import cv2
import h5py
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import *
from collections import Counter
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import re
import time
import random

from keras.models import *
import keras.backend as K
from make_parallel import make_parallel

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Using TensorFlow backend.


In [7]:
characters = u'0123456789()+-*/=君不见黄河之水天上来奔流到海复回烟锁池塘柳深圳铁板烧; '

n_len = 45
rnn_length = 110
n, width, height, n_class, channels = 100000, 900, 81, len(characters), 3

In [4]:
def decode(out):
    return ''.join([characters[x] for x in out if x < n_class-1 and x > -1])

def disp3(index):
    s = decode(out[index])
    plt.figure(figsize=(16, 4))
    plt.imshow(X[index].transpose(1, 0, 2))
    plt.title('pred:%s'%s)

def disp2(img):
    cv2.imwrite('a.png', img)
    return Image('a.png')

def disp(img, txt=None, first=False):
    global index
    if first:
        index = 1
        plt.figure(figsize=(16, 9))
    else:
        index += 1
    plt.subplot(4, 1, index)
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img[:,:,::-1])
    if txt:
        plt.title(txt)

# 读取测试集

In [5]:
X = np.zeros((n, width, height, channels), dtype=np.uint8)

for i in tqdm(range(n)):
    img = cv2.imread('crop_split2_test/%d.png'%i).transpose(1, 0, 2)
    a, b, _ = img.shape
    X[i, :a, :b] = img

100%|██████████| 100000/100000 [01:38<00:00, 1013.91it/s]


# 预测


In [8]:
z = '0.997754'

base_model = load_model('model_346_split2_3_%s.h5' % z)

base_model2 = make_parallel(base_model, 4)
y_pred = base_model2.predict(X, batch_size=500, verbose=1)
out = K.get_value(K.ctc_decode(y_pred[:,2:], input_length=np.ones(y_pred.shape[0])*rnn_length)[0][0])[:, :n_len]

100000/100000 [==============================] - 358s   


In [25]:
ss = map(decode, out)

vals = []
errs = []
errsid = []
for i in tqdm(range(100000)):
    val = ''
    try:
        a = ss[i].split(';')
        s = a[-1]
        for x in a[:-1]:
            x, c = x.split('=')
            s = s.replace(x, c+'.0')
        val = '%.2f' % eval(s)
    except:
#         disp3(i)
        errs.append(ss[i])
        errsid.append(i)
        ss[i] = ''
    
    vals.append(val)
    
with open('result_%s.txt' % z, 'w') as f:
    f.write('\n'.join(map(' '.join, list(zip(ss, vals)))).encode('utf-8'))
    
print len(errs)
print 1-len(errs)/100000.

100%|██████████| 100000/100000 [00:01<00:00, 87655.64it/s]


22
0.99978


In [26]:
z = '0.997559'

base_model = load_model('model_346_split2_3_%s.h5' % z)

base_model2 = make_parallel(base_model, 4)
y_pred = base_model2.predict(X, batch_size=500, verbose=1)
out = K.get_value(K.ctc_decode(y_pred[:,2:], input_length=np.ones(y_pred.shape[0])*rnn_length)[0][0])[:, :n_len]

100000/100000 [==============================] - 277s   


In [28]:
ss = map(decode, out)

vals = []
errs = []
errsid = []
for i in tqdm(range(100000)):
    val = ''
    try:
        a = ss[i].split(';')
        s = a[-1]
        for x in a[:-1]:
            x, c = x.split('=')
            s = s.replace(x, c+'.0')
        val = '%.2f' % eval(s)
    except:
#         disp3(i)
        errs.append(ss[i])
        errsid.append(i)
        ss[i] = ''
    
    vals.append(val)
    
with open('result_%s_%d.txt' % (z, len(errs)), 'w') as f:
    f.write('\n'.join(map(' '.join, list(zip(ss, vals)))).encode('utf-8'))
    
print len(errs)
print 1-len(errs)/100000.

100%|██████████| 100000/100000 [00:01<00:00, 87565.60it/s]

20
0.9998
